In [12]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:

#load csv file
import pandas as pd # Import the pandas library with the alias 'pd'
df = pd.read_csv('/content/articles.csv',encoding ='cp1252')
df.head(2)

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bell’s Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive


In [14]:
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    return text

In [18]:
# Apply the cleaning function to the relevant columns
df['clean_text'] = df['Full_Article'].apply(clean_text)

In [16]:
!pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [17]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained SentenceBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the clean_text using SentenceBERT
embeddings = model.encode(df['clean_text'].tolist())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
from sklearn.impute import SimpleImputer

# Create an imputer (mean strategy)
imputer = SimpleImputer(strategy='mean')

In [23]:
# Apply the imputer to the training data
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [24]:
# Check for NaN values in embeddings
print(np.isnan(embeddings).sum())



0


In [26]:
# Convert to float just in case
X_train = X_train.astype(float)
X_test = X_test.astype(float)


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [20]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['Article_Type'], test_size=0.2, random_state=42)

In [31]:
# Check for NaN values in y_train
print(y_train.isna().sum())


1


In [32]:
# Fill NaN values in y_train with a placeholder
y_train.fillna('Unknown', inplace=True)


In [33]:
from sklearn.tree import DecisionTreeClassifier

# Try with a Decision Tree model
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)



In [39]:
y_pred_train_dt = classifier.predict(X_train)
y_pred_tes_dt = classifier.predict(X_test)

In [40]:
# Evaluate the model
print(accuracy_score(y_train, y_pred_train_dt))
print()
accuracy = accuracy_score(y_test, y_pred_tes_dt)
print(f'Accuracy with Decision Tree: {accuracy}')
print(classification_report(y_test, y_pred_tes_dt))

0.9489322191272052

Accuracy with Decision Tree: 0.9333333333333333
              precision    recall  f1-score   support

  Commercial       0.96      0.98      0.97       219
  Executives       0.00      0.00      0.00         1
   Financing       0.00      0.00      0.00         1
    Military       0.83      0.90      0.86        42
      Others       0.00      0.00      0.00         6
    Training       0.00      0.00      0.00         1

    accuracy                           0.93       270
   macro avg       0.30      0.31      0.30       270
weighted avg       0.90      0.93      0.92       270



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# model 2 logistic regression

In [35]:
from sklearn.linear_model import LogisticRegression

In [41]:
# Train a Logistic Regression model
classifier1 = LogisticRegression(max_iter=1000)
classifier1.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [44]:
# Predict on the test set
y_pred_train_log = classifier1.predict(X_train)
y_pred_test_log = classifier1.predict(X_test)

In [45]:
# Evaluate the model
print(accuracy_score(y_train, y_pred_train_log))
print()
accuracy = accuracy_score(y_test, y_pred_test_log)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred_test_log))

0.9489322191272052

Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

  Commercial       0.96      0.98      0.97       219
  Executives       0.00      0.00      0.00         1
   Financing       0.00      0.00      0.00         1
    Military       0.83      0.90      0.86        42
      Others       0.00      0.00      0.00         6
    Training       0.00      0.00      0.00         1

    accuracy                           0.93       270
   macro avg       0.30      0.31      0.30       270
weighted avg       0.90      0.93      0.92       270



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm.fit(X_train, y_train)

RandomForestClassifier()

In [49]:
y_pred_train_rfm = rfm.predict(X_train)
y_pred_test_rfm = rfm.predict(X_test)

In [50]:
print(classification_report(y_train, y_pred_train_rfm))
print()
print(classification_report(y_test, y_pred_test_rfm))

                    precision    recall  f1-score   support

        Commercial       1.00      1.00      1.00       895
        Executives       1.00      1.00      1.00         2
         Financing       1.00      1.00      1.00         1
          Military       1.00      1.00      1.00       166
            Others       1.00      1.00      1.00         9
Support & Services       1.00      1.00      1.00         1
          Training       1.00      1.00      1.00         2
           Unknown       1.00      1.00      1.00         1

          accuracy                           1.00      1077
         macro avg       1.00      1.00      1.00      1077
      weighted avg       1.00      1.00      1.00      1077


              precision    recall  f1-score   support

  Commercial       0.88      0.99      0.93       219
  Executives       0.00      0.00      0.00         1
   Financing       0.00      0.00      0.00         1
    Military       0.85      0.40      0.55        42
     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
print(accuracy_score(y_train, y_pred_train_rfm))
print()
print(accuracy_score(y_test, y_pred_test_rfm))

1.0

0.8777777777777778


In [52]:
import pickle

In [53]:

# Save the model to a file
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rfm, file)

In [54]:

# Load the model from the file
with open('random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)





In [55]:
y_pred = loaded_model.predict(X_test)

In [56]:
# Load new data
new_data = pd.read_csv('/content/unknown_articles.csv')


In [71]:
import requests

def extract_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    heading = soup.title.string
    article = soup.get_text(separator=' ', strip=True)
    return heading, article


In [72]:
# Load new data
new_data = pd.read_csv('/content/unknown_articles.csv')


In [75]:
# Clean and vectorize the new data
new_data['clean_text'] = new_data['Article.URL'].apply(clean_text)
new_embeddings = model.encode(new_data['clean_text'].tolist())

<ipython-input-14-7d51ff552f8c>:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [77]:
import requests
from bs4 import BeautifulSoup

def clean_text(text):
    # Check if the input looks like a URL
    if text.startswith(('http://', 'https://')):
        # Fetch the content from the URL
        response = requests.get(text)
        if response.status_code == 200:
            text = response.text
        else:
            return ''  # Return empty string if URL fetch fails

    # Use BeautifulSoup to parse HTML content
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
